In [1]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
sns.set_style('white')
plt.set_cmap('gist_gray')


# We also install a package to read NiFTI files
import nibabel as nib

import torch
import torchvision.models as models
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import lr_scheduler
import torch.utils.data as utils
import time
import copy

In [2]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print('using device:', device)

using device: cuda


In [12]:
#Saves the tensor
# torch.save(new_img_list,"../../../data/data_preserved.pt")
# x = torch.load("../../../data/data_preserved.pt")

# print(x.shape)
x = torch.load("../../../data/data_64_3.pt")

print(x.shape)

torch.Size([536, 64, 64, 64])


In [13]:
y = torch.load("../../../data/ages.pt")
n = y.shape[0]
y = y.reshape((n,1))
print(y.shape)

torch.Size([536, 1])


In [14]:
dataset = utils.TensorDataset(x, y)

In [15]:
train_indices = [i for i in range(n) if (i % 5 == 0 or i % 5 == 2 or i % 5 == 4)]
val_indices = [i for i in range(n) if i % 5 == 3]
test_indices = [i for i in range(n) if i not in train_indices and i not in val_indices]

print("There are %d (%.2f) training data points, %d (%.2f) validation data points,\and %d (%.2f) test data points" %(len(train_indices),len(train_indices)/n,len(val_indices),\
                                                                                                                len(val_indices)/n,len(test_indices),len(test_indices)/n))
train_dataset = utils.TensorDataset(x[train_indices],y[train_indices])
val_dataset = utils.TensorDataset(x[val_indices],y[val_indices])      
dataloaders = {}
dataloaders['train'] = torch.utils.data.DataLoader(train_dataset, batch_size=4,
                                             shuffle=True,num_workers=4)
dataloaders['val'] = torch.utils.data.DataLoader(val_dataset, batch_size=4,
                                             shuffle=True,num_workers=4)

There are 322 (0.60) training data points, 107 (0.20) validation data points,nd 107 (0.20) test data points


In [16]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_offset = float('inf')

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_offset = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device,dtype = torch.float)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_offset += torch.sum(torch.abs(outputs - labels.data))

            if phase == 'train':
                epoch_loss = running_loss / (n * 0.6)
                epoch_offset = running_offset / (n * 0.6)
               
            else:
                epoch_loss = running_loss / (n * 0.2)
                epoch_offset = running_offset / (n * 0.2)
            

            print('{} Loss: {:.4f} Diff: {:.4f}'.format(
                phase, epoch_loss, epoch_offset))

            # deep copy the model
            if phase == 'val' and epoch_offset < best_offset:
                best_offset = epoch_offset
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Diff: {:4f}'.format(best_offset))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [17]:
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

In [21]:

# model = nn.Sequential(
#     nn.Conv2d(in_channels=256, out_channels=channel_1, kernel_size=5, stride=1, \
#               padding=2, bias=True),
#     nn.Conv2d(in_channels=channel_1, out_channels=channel_2, kernel_size=5, stride=1, \
#               padding=2, bias=True),
#     nn.ReLU(),
#     nn.MaxPool2d(kernel_size = 2, stride=2),
#     nn.Conv2d(in_channels=channel_2, out_channels=channel_1, kernel_size=3, stride=1, \
#               padding=1,bias=True),
#     nn.Conv2d(in_channels=channel_1, out_channels=channel_2, kernel_size=3, stride=1, \
#               padding=1,bias=True),
#     nn.ReLU(),
#     nn.MaxPool2d(kernel_size = 2, stride=2),
#     nn.Conv2d(in_channels=channel_2, out_channels=channel_1, kernel_size=3, stride=1, \
#               padding=1,bias=True),
#     nn.Conv2d(in_channels=channel_1, out_channels=channel_2, kernel_size=3, stride=1, \
#               padding=1,bias=True),
#     nn.ReLU(),
#     nn.MaxPool2d(kernel_size = 2, stride=2),
#     Flatten(),
#     nn.Linear(channel_2 * 1 * 18, 1),
# )

model = nn.Sequential(
    nn.Conv2d(64, 32, 5, 1, padding=2, bias=True).to(device),
    nn.Conv2d(32, 32, 5, 1, padding=2, bias=True).to(device),
    nn.BatchNorm2d(32).to(device),
    nn.AvgPool2d(kernel_size = 2, stride=2).to(device),
    nn.Conv2d(32, 64,5,1,padding=2,bias=True).to(device),
    nn.Conv2d(64, 64,5,1,padding=2,bias=True).to(device),
    nn.BatchNorm2d(64).to(device),
    nn.AvgPool2d(kernel_size = 2, stride=2).to(device),
    nn.Conv2d(64, 128,3,1,padding=1,bias=True).to(device),
    nn.Conv2d(128, 128,3,1,padding=1,bias=True).to(device),
    nn.BatchNorm2d(128).to(device),
    nn.AvgPool2d(kernel_size = 2, stride=2).to(device),
    nn.Conv2d(128, 256,3,1,padding=1,bias=True).to(device),
    nn.Conv2d(256, 256,3,1,padding=1,bias=True).to(device),
    nn.BatchNorm2d(256).to(device),
    nn.AvgPool2d(kernel_size = 2, stride=2).to(device),
    nn.Conv2d(256, 512,3,1,padding=1,bias=True).to(device),
    nn.Conv2d(512, 512,3,1,padding=1,bias=True).to(device),
    nn.BatchNorm2d(512).to(device),
    nn.AvgPool2d(kernel_size = 2, stride=2).to(device),
    Flatten().to(device),
    nn.Linear(512*1*2*2, 1).to(device),
)

In [22]:
# model_ft = models.resnet18(pretrained=True)

# num_ftrs = model_ft.fc.in_features
# model_ft.fc = nn.Linear(num_ftrs, 1)

# model_ft = model_ft.to(device)

criterion = nn.MSELoss()

# Observe that all parameters are being optimized
# optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
optimizer_ft = optim.Adam(model.parameters(), lr=0.001)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [23]:
model = train_model(model, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=25)

Epoch 0/24
----------
train Loss: 760.6533 Diff: 21.6383
val Loss: 2848.0234 Diff: 49.6464

Epoch 1/24
----------
train Loss: 230.8124 Diff: 12.1872
val Loss: 213.9583 Diff: 12.4234

Epoch 2/24
----------
train Loss: 197.2146 Diff: 11.4923
val Loss: 330.0891 Diff: 14.8748

Epoch 3/24
----------
train Loss: 212.5860 Diff: 11.9435
val Loss: 249.7138 Diff: 12.9055

Epoch 4/24
----------
train Loss: 197.5732 Diff: 11.5805
val Loss: 208.5511 Diff: 12.0164

Epoch 5/24
----------
train Loss: 197.9519 Diff: 11.2985
val Loss: 205.6681 Diff: 12.2909

Epoch 6/24
----------
train Loss: 171.7247 Diff: 10.8559
val Loss: 190.3245 Diff: 11.4894

Epoch 7/24
----------
train Loss: 166.1309 Diff: 10.6432
val Loss: 189.9448 Diff: 11.5321

Epoch 8/24
----------
train Loss: 175.3148 Diff: 10.7761
val Loss: 194.4403 Diff: 11.6914

Epoch 9/24
----------
train Loss: 168.4462 Diff: 10.5314
val Loss: 194.1175 Diff: 11.6177

Epoch 10/24
----------
train Loss: 159.2187 Diff: 10.1914
val Loss: 198.4221 Diff: 11.705

In [12]:
def check_error(data,labels,model):
    print('Checking error on test set')   
    total_abs_err = 0.0
    num_samples = data.shape[0]
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        data = data.to(device=device, dtype=dtype)  # move to device, e.g. GPU
        labels = labels.to(device=device,dtype=dtype)
        scores = model(data)
        print((labels - scores).shape)
        total_abs_err += torch.abs(labels - scores).sum()
        mean_abs_err = float(total_abs_err) / num_samples
        print('Got error %.4f' % (mean_abs_err))

In [13]:
# y = torch.reshape(y,(n,1))
check_error(x[val_indices,:], y[val_indices],model)

Checking error on test set
torch.Size([107, 1])
Got error 12.1053


In [14]:
torch.save(model,"best_2d_model_preserved.pth")